In [1]:
import pandas as pd
import datetime

In [3]:
#de la carpeta Data estraermos el .csv que contine la data ya limpia.
Data=pd.read_csv("../Informes/Data_Limpia.csv")
Data.head()

,CaptureDate,Aerolinea,Fecha,Departure,Arrive,Hora_Salida,Hora_LLegada,Price,Tipo
0,2023-04-18,Latam,2023-04-23,SMR,MDE,20:35:00,21:51:00,993760,Directo
1,2023-04-18,Latam,2023-04-23,SMR,MDE,07:18:00,10:44:00,214790,1_parada
2,2023-04-18,Latam,2023-04-23,SMR,MDE,07:18:00,11:11:00,214790,1_parada
3,2023-04-18,Latam,2023-04-23,SMR,MDE,07:18:00,12:56:00,214790,1_parada
4,2023-04-18,Latam,2023-04-23,SMR,MDE,07:43:00,11:11:00,227280,1_parada


In [3]:
Data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7616 entries, 0 to 7615
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   CaptureDate   7616 non-null   object
 1   Aerolinea     7608 non-null   object
 2   Fecha         7616 non-null   object
 3   Departure     7616 non-null   object
 4   Arrive        7616 non-null   object
 5   Hora_Salida   7616 non-null   object
 6   Hora_LLegada  7616 non-null   object
 7   Price         7616 non-null   int64 
 8   Tipo          7616 non-null   object
dtypes: int64(1), object(8)
memory usage: 535.6+ KB


In [4]:
#Transformar la data a los type necesarios para la creacion de las nuevas variables
Data['CaptureDate']=pd.to_datetime(Data['CaptureDate'])
Data['Fecha']=pd.to_datetime(Data['Fecha'])

Data['Hora_Salida'] = pd.to_datetime(Data['Hora_Salida'], format='%H:%M:%S').dt.time
Data['Hora_LLegada'] = pd.to_datetime(Data['Hora_LLegada'], format='%H:%M:%S').dt.time

#### Creacion de las nuevas variables
derivadas de las variables principales

- Fechas: Se deriva como Departure_Day, Departure_Month, Departure_Year, Weekday , Season, Holidays
- Hora_Salida: en Departure_time
- DepartureDate y Fecha: su difenrecia crea la variable Consultation_period

In [5]:
# Extraer las columnas requeridas
Data['Departure_Day'] = Data['Fecha'].dt.day
Data['Departure_Month'] = Data['Fecha'].dt.month
Data['Departure_Year'] = Data['Fecha'].dt.year
Data['Weekday'] = Data['Fecha'].dt.weekday


In [6]:
# Definir una función que devuelva "día" o "noche" en función de la hora
def calcular_horario(hora):
    """
    This function takes a time input and returns a string indicating whether it is in the morning,
    afternoon, evening, or late night.
    
    :param hora: The input parameter "hora" is expected to be a time object in the format of "HH:MM:SS"
    (hours, minutes, seconds) or a string that can be parsed into a time object
    :return: a string indicating the time of day based on the input time. The possible return values are
    "Madrugada" (dawn), "Mañana" (morning), "Tarde" (afternoon), or "Noche" (night).
    """
    hora_objeto = hora 
    if hora_objeto < datetime.time(6,0,0):
        return "Madrugada"
    elif hora_objeto < datetime.time(12,0,0):
        return "Mañana"
    elif hora_objeto < datetime.time(18,0,0):
        return "Tarde"
    else:
        return "Noche"

# Crear una nueva columna "momento_del_dia" que indique si la observación se realizó durante el día o la noche
Data['Departure_time'] = Data['Hora_Salida'].apply(lambda x : calcular_horario(x))

In [7]:
from datetime import datetime
# Definimos las fechas comunes para vacacionar en Colombia
vacaciones = [('12-15', '01-15'), ('06-20', '07-20'), ('03-21', '04-07'), ('05-01', '05-03')]

# Función para verificar si una fecha está dentro de las fechas comunes para vacacionar
def esta_en_vacaciones(fecha):
    for inicio, fin in vacaciones:
        fecha_inicio = datetime.strptime(f'{fecha.year}-{inicio}', '%Y-%m-%d').date()
        fecha_fin = datetime.strptime(f'{fecha.year}-{fin}', '%Y-%m-%d').date()
        if fecha_inicio <= fecha.date() <= fecha_fin:
            return True
    return False

# Creamos la columna indicando si la fecha está en las fechas comunes para vacacionar
Data['Holidays'] = Data['Fecha'].apply(lambda x: esta_en_vacaciones(x))

In [11]:
Data['Consultation_period']=(Data['Fecha'] - Data['CaptureDate']).dt.days

In [12]:
Data

,CaptureDate,Aerolinea,Fecha,Departure,Arrive,Hora_Salida,Hora_LLegada,Price,Tipo,Departure_Day,Departure_Month,Departure_Year,Weekday,Departure_time,Holidays,Consultation_period
0,2023-04-18,Latam,2023-04-23,SMR,MDE,20:35:00,21:51:00,993760,Directo,23,4,2023,6,Noche,False,5
1,2023-04-18,Latam,2023-04-23,SMR,MDE,07:18:00,10:44:00,214790,1_parada,23,4,2023,6,Mañana,False,5
2,2023-04-18,Latam,2023-04-23,SMR,MDE,07:18:00,11:11:00,214790,1_parada,23,4,2023,6,Mañana,False,5
3,2023-04-18,Latam,2023-04-23,SMR,MDE,07:18:00,12:56:00,214790,1_parada,23,4,2023,6,Mañana,False,5
4,2023-04-18,Latam,2023-04-23,SMR,MDE,07:43:00,11:11:00,227280,1_parada,23,4,2023,6,Mañana,False,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7611,2023-04-25,Wingo,2023-06-17,CTG,BOG,12:42:00,14:07:00,216000,Directo,17,6,2023,5,Tarde,False,53
7612,2023-04-25,Wingo,2023-06-21,CTG,BOG,21:57:00,23:22:00,216000,Directo,21,6,2023,2,Noche,True,57
7613,2023-04-25,Wingo,2023-06-23,CTG,BOG,14:49:00,16:14:00,216000,Directo,23,6,2023,4,Tarde,True,59
7614,2023-04-25,Wingo,2023-06-23,CTG,BOG,18:52:00,20:17:00,216000,Directo,23,6,2023,4,Noche,True,59


In [13]:
Data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7616 entries, 0 to 7615
Data columns (total 16 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   CaptureDate          7616 non-null   datetime64[ns]
 1   Aerolinea            7608 non-null   object        
 2   Fecha                7616 non-null   datetime64[ns]
 3   Departure            7616 non-null   object        
 4   Arrive               7616 non-null   object        
 5   Hora_Salida          7616 non-null   object        
 6   Hora_LLegada         7616 non-null   object        
 7   Price                7616 non-null   int64         
 8   Tipo                 7616 non-null   object        
 9   Departure_Day        7616 non-null   int64         
 10  Departure_Month      7616 non-null   int64         
 11  Departure_Year       7616 non-null   int64         
 12  Weekday              7616 non-null   int64         
 13  Departure_time       7616 non-nul